In [1]:
from kdm.models import KDMSequentialJointClassModel, KDMGraphModel
import kdm
from sklearn.model_selection import train_test_split
from sklearn.datasets import make_moons
import matplotlib.pyplot as plt
import numpy as np
from keras import metrics
from keras import losses
from keras import optimizers
from keras.models import Model
from keras.layers import Input, Dense
import keras
from pandas import read_csv, DataFrame
import os
import keras

os.environ["KERAS_BACKEND"] = "tensorflow"


In [2]:
df = read_csv('asia_cancer.csv')

In [3]:
df.head()

,A,S,T,L,B,E,X,D
0,no,yes,no,no,yes,no,no,yes
1,no,yes,no,no,no,no,no,no
2,no,no,yes,no,no,yes,yes,yes
3,no,no,no,no,yes,no,no,yes
4,no,no,no,no,no,no,no,yes


In [4]:
# parse all columns to 1 and 0
df = df.applymap(lambda x: 1 if x == 'yes' else 0)

In [5]:
df.head()

,A,S,T,L,B,E,X,D
0,0,1,0,0,1,0,0,1
1,0,1,0,0,0,0,0,0
2,0,0,1,0,0,1,1,1
3,0,0,0,0,1,0,0,1
4,0,0,0,0,0,0,0,1


In [20]:

encoded_size = 2
dim_y = 1
encoder = keras.layers.Identity()
n_comp = 5000
X = df[['A', 'S']].astype(np.float32)
y = df['D']
  
nodes = [
    {'name': 'input_A', 'type': 'input', 'dim_x': 1, 'dim_y': 1},
    {'name': 'input_S', 'type': 'input', 'dim_x': 1, 'dim_y': 1},
    {'name': 'T', 'dim_x': 1, 'dim_y': 1},
    {'name': 'L', 'dim_x': 1, 'dim_y': 1},
    {'name': 'B', 'dim_x': 1, 'dim_y': 1},
    {'name': 'E', 'dim_x': 1, 'dim_y': 1},
    {'name': 'output', 'dim_x': 1, 'dim_y': 1}
]

edges = [
    ('input_A', 'T'),
    ('T', 'E'),
    ('input_S', 'L'),
    ('input_S', 'B'),
    ('L', 'E'),
    ('E', 'output'),
    ('B', 'output')
]

In [21]:
model = KDMGraphModel(
    encoded_size=encoded_size,
    dim_y=dim_y,
    encoder=encoder,  # You can define an encoder if needed
    n_comp=n_comp,
    sigma=0.5,
    nodes=nodes,
    edges=edges
)

In [22]:
model.compile(optimizer=optimizers.Adam(learning_rate=5e-5),
                  loss="binary_crossentropy", metrics=['accuracy'])

In [23]:
X

,A,S
0,0.0,1.0
1,0.0,1.0
2,0.0,0.0
3,0.0,0.0
4,0.0,0.0
...,...,...
4995,0.0,0.0
4996,0.0,0.0
4997,0.0,0.0
4998,0.0,1.0


In [24]:
np.array(X)

array([[0., 1.],
       [0., 1.],
       [0., 0.],
       ...,
       [0., 0.],
       [0., 1.],
       [0., 1.]], dtype=float32)

In [25]:
X_dict = {f'input_{col}': X[[col]].values for col in X.columns}

In [26]:
print(X_dict)

{'input_A': array([[0.],
       [0.],
       [0.],
       ...,
       [0.],
       [0.],
       [0.]], dtype=float32), 'input_S': array([[1.],
       [1.],
       [0.],
       ...,
       [0.],
       [1.],
       [1.]], dtype=float32)}


In [27]:
history = model.fit(
    X_dict,
    y,  # Your training labels
    batch_size=4,
    epochs=300,  # Start with 100 epochs and use early stopping
    verbose=1,  # Detailed logging
    validation_split=0.1,  # Explicit validation data
    shuffle=True  # Shuffle the data
)

Epoch 1/300
merged probs
Tensor("Reshape:0", shape=(4, 1), dtype=float32)
Node: E, rho_x shape: (4, 1, 2), rho_x_values: Tensor("concat_2:0", shape=(4, 1, 2), dtype=float32)
merged probs
Tensor("Reshape_1:0", shape=(4, 1), dtype=float32)
Node: output, rho_x shape: (4, 1, 2), rho_x_values: Tensor("concat_3:0", shape=(4, 1, 2), dtype=float32)
merged probs
Tensor("kdm_graph_model_1_1/Reshape:0", shape=(4, 1), dtype=float32)
Node: E, rho_x shape: (4, 1, 2), rho_x_values: Tensor("kdm_graph_model_1_1/concat_2:0", shape=(4, 1, 2), dtype=float32)
merged probs
Tensor("kdm_graph_model_1_1/Reshape_1:0", shape=(4, 1), dtype=float32)
Node: output, rho_x shape: (4, 1, 2), rho_x_values: Tensor("kdm_graph_model_1_1/concat_3:0", shape=(4, 1, 2), dtype=float32)
merged probs
Tensor("kdm_graph_model_1_1/Reshape:0", shape=(4, 1), dtype=float32)
Node: E, rho_x shape: (4, 1, 2), rho_x_values: Tensor("kdm_graph_model_1_1/concat_2:0", shape=(4, 1, 2), dtype=float32)
merged probs
Tensor("kdm_graph_model_1_1/Res

KeyboardInterrupt: 

In [ ]:
results = model.evaluate(X_dict, y)
print(f"Loss: {results[0]}, Accuracy: {results[1]}")